In [ ]:
import numpy as np              
from time import time
import os
from os import listdir
import cv2
import scipy.linalg as sg
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image 
from math import sqrt
from shutil import copy2 as copy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Funciones

In [ ]:
def integral(ch):
    s=ch
    for i in range(ch.ndim):
        s=s.cumsum(axis = i)
        
    return s

In [ ]:
def pintar(cov):
    new_cov=np.copy(cov)
    for i in range(cov.shape[1]):
        for j in range(cov.shape[0]):
            if(new_cov[i,j]<=0):
                new_cov[i,j]=-1*(new_cov[i,j]/np.min(cov))
            else:
                new_cov[i,j]=new_cov[i,j]/np.max(cov)
    return new_cov

In [ ]:
def tr_text(dir):
    temp=''
    with open(dir) as fp:  
        line = fp.readline()
        cnt = 1
        while line:
            line = fp.readline()
            temp= temp+line[:-1]
            cnt += 1
    return(temp)

In [ ]:
def dirr_carps(url):
    list_elements_1,list_elements_2=[],[]
    for carp in listdir(url):
        list_elements_1.append(url+carp)
        list_elements_2.append(carp)
    return sorted(list_elements_1),sorted(list_elements_2)

In [ ]:
def pintar(cov):
    new_cov=np.copy(cov)
    for i in range(cov.shape[1]):
        for j in range(cov.shape[0]):
            if(new_cov[i,j]<=0):
                new_cov[i,j]=-1*(new_cov[i,j]/np.min(cov))
            else:
                new_cov[i,j]=new_cov[i,j]/np.max(cov)
    return new_cov

In [ ]:
def cortar(x):
    #x = trayectorias en texto            
    tr=[]
    gtr=[]
    count = 0
    n = 0
    for i in range(0,len(x)):
        if(x[i]==' '):
            count=count+1
            tr.append(float(x[n:i]))
            n=i
        if(count==34):
            gtr.append(tr)
            count=0
            tr=[]
    return gtr

In [ ]:
def int_great_array_renew(convimg,frame):
    ch_array=np.array([convimg[0],convimg[1],convimg[2],convimg[3],convimg[4],convimg[5],convimg[6],convimg[7],convimg[8],convimg[9],convimg[10],convimg[11],convimg[12],convimg[13],convimg[14],convimg[15],convimg[16]])#,convimg[17]])#,convimg[18],convimg[19]])#,convimg[20],convimg[21],convimg[22],convimg[23],convimg[24],convimg[25],convimg[26],convimg[27],convimg[28],convimg[29],convimg[30],convimg[31],convimg[32],convimg[33],convimg[34],convimg[35],convimg[36],convimg[37],convimg[38],convimg[39],convimg[40],convimg[41],convimg[42],convimg[43],convimg[44],convimg[45],convimg[46],convimg[47],convimg[48]])#,convimg[49],convimg[50],convimg[51],convimg[52],convimg[53],convimg[54],convimg[55],convimg[56],convimg[57],convimg[58],convimg[59],convimg[60],convimg[61],convimg[62],convimg[63]])
    d=ch_array.shape[0]
    W=ch_array.shape[1]
    H=ch_array.shape[2]
    #print(d,W,H)
    #Tensor P
    tensor_p=np.zeros((d,W,H))
    for i in range(d):
        tensor_p[i]= integral(ch_array[i])
        
    #Tensor Q    
    nmatrices=((d * (d + 1)) / 2 )
    tensor_q=np.zeros((int(nmatrices),W,H))              
    pos=0
    for i in range(d):
        for j in range(i,d):
            mult=ch_array[i]*ch_array[j]
                                 #producto elemento a elemento entre tipos arrays
            #np.save('/content/drive/My Drive/BIVL2AB/Analisis de movimiento y vision por computador/Project/mobilnetfeatures/TensoresT/frame'+str(frame)+'_'+str(pos)+'q',integral(mult))
            tensor_q[pos]=integral(mult)
            pos=pos+1
       
    
    return ch_array,tensor_p,tensor_q

In [ ]:
def covariance(great_p,great_q,T_area):
    
    n_frames = great_p.shape[0]
    d = great_p.shape[1]
    m_cov=np.zeros((d,d))
    pos = 0
    y1=T_area[0][0]
    x1=T_area[0][1]
    y2=T_area[0][2]
    x2=T_area[0][3]
    col=x2-x1
    fil=y2-y1
    n1=(1/(((fil)*(col))*n_frames-1))
    n2=(1/((fil)*(col)*n_frames))
    
    try:
      for i in range(d):
          #print(i)
          for j in range(i,d):
                  p_1=0
                  p_2=0
                  q=0
                  #print(pos)
                  for k in range(n_frames):
                      
                      y1=T_area[k][0]-1
                      x1=T_area[k][1]-1
                      y2=T_area[k][2]
                      x2=T_area[k][3]
                      
                      #temp_q=np.loadtxt('/content/drive/My Drive/BIVL2AB/Analisis de movimiento y vision por computador/Project/mobilnetfeatures/TensoresT/frame'+str(great_q[k])+'_'+str(pos)+'q')
                      #q=temp_q[y1,x1]+temp_q[y2,x2]-temp_q[y1,x2]-temp_q[y2,x1]+q
                      q=great_q[k,0,pos]+great_q[k,1,pos]-great_q[k,2,pos]-great_q[k,3,pos]+q

                      #p_1=great_p[k,0,i]+great_p[k,1,i]-great_p[k,2,i]-great_p[k,3,i]+p_1
                      #p_2=great_p[k,0,j]+great_p[k,1,j]-great_p[k,2,j]-great_p[k,3,j]+p_2

                      #q=temp_q[pos,y1,x1]+temp_q[pos,y2,x2]-temp_q[pos,y1,x2]-temp_q[pos,y2,x1]+q            
                      #p_1=temp_p[i,y1,x1]+temp_p[i,y2,x2]-temp_p[i,y1,x2]-temp_p[i,y2,x1]+p_1
                      #p_2=temp_p[j,y1,x1]+temp_p[j,y2,x2]-temp_p[j,y1,x2]-temp_p[j,y2,x1]+p_2


                      #q=great_q[k,pos,y1,x1]+great_q[k,pos,y2,x2]-great_q[k,pos,y1,x2]-great_q[k,pos,y2,x1]+q
                      p_1=great_p[k,i,y1,x1]+great_p[k,i,y2,x2]-great_p[k,i,y1,x2]-great_p[k,i,y2,x1]+p_1
                      p_2=great_p[k,j,y1,x1]+great_p[k,j,y2,x2]-great_p[k,j,y1,x2]-great_p[k,j,y2,x1]+p_2
                      
                  pos=pos+1
                  m_cov[i,j]=(n1)*(q-(n2*p_1*p_2))
                  
                  if(i!=j):
                      m_cov[j,i]=m_cov[i,j]
      flag = 1
    except:
      flag = None
      m_cov = np.zeros((d,d))
    
    return m_cov,flag

In [ ]:
def covarianza(Tensor,area):
  trayectorias,frames,c,d,e=Tensor.shape
  col=area*2
  fil=area*2
  n1=(1/(((fil)*(col))*frames-1))
  n2=(1/((fil)*(col)*frames))
  Covarianzas=[]
  for i in range(trayectorias):
    #m_cov=np.zeros((d,d))
    p_1 = np.sum(Tensor[i,:,:,:,0],axis=0)
    p_2 = np.sum(Tensor[i,:,:,:,1],axis=0)
    q = np.sum(Tensor[i,:,:,:,2],axis=0)
    m_cov=(n1)*(q-(n2*p_1*p_2))
    Covarianzas.append(m_cov)

  return Covarianzas

In [ ]:
def Tensor_covarianza(great_q,great_p,T_area,Tensor):
    #T_area tiene como argumentos una 4-tupla con y1,x1,y2,x2 y por ultimo j, que es el numero de la trayectoria.
    #great_q es mi tensor q recien cargado
    #great_p es el tensor p que tengo en memoria
    d = great_p.shape[0]
    m_cov=np.zeros((d,d))
    pos = 0
    y1=T_area[0][0][0]
    x1=T_area[0][0][1]
    y2=T_area[0][0][2]
    x2=T_area[0][0][3]
    col=x2-x1
    fil=y2-y1

    for k in range(len(T_area)):
      y1=T_area[k][0][0]
      x1=T_area[k][0][1]
      y2=T_area[k][0][2]
      x2=T_area[k][0][3]
      trayectoria=T_area[k][1][0]
      frame=T_area[k][1][1]-1
      pos = 0
      
      for i in range(d):
        for j in range(i,d):
            
            q=great_q[pos,y1,x1]+great_q[pos,y2,x2]-great_q[pos,y1,x2]-great_q[pos,y2,x1] 
            p_1=great_p[i,y1,x1]+great_p[i,y2,x2]-great_p[i,y1,x2]-great_p[i,y2,x1]
            p_2=great_p[j,y1,x1]+great_p[j,y2,x2]-great_p[j,y1,x2]-great_p[j,y2,x1]      
            
            pos=pos+1
            Tensor[trayectoria,frame,i,j,0]=q
            Tensor[trayectoria,frame,i,j,1]=p_1
            Tensor[trayectoria,frame,i,j,2]=p_2
            if(i!=j):
              Tensor[trayectoria,frame,j,i,:]=Tensor[trayectoria,frame,i,j,:]       
    
    return Tensor

In [ ]:
def video_covs(path,trayectorias,area,path2):
    feat,f,great_p,great_q,Q=[],[],[],[],[]
    Covarianzas=[]
    images = [path+image for image in os.listdir(path)]
    n_images = len(images)

    if isinstance(sqrt(n_images)%1,float):limits = int(sqrt(n_images))+1 
    else:limits = int(sqrt(n_images))  
    for i,image in enumerate(images):
        im = Image.open(image)
        b='/home/oskkr/BIV2LAB/parkinsonfeatures/PK-ojos/'+path2+'/frame'+str(i+1)+'.npy'
    
        feat.append(b)
             
        x,p,q=int_great_array_renew(np.load(feat[i]),i+1)
        #p=np.load('/content/drive/My Drive/BIVL2AB/Analisis de movimiento y vision por computador/Project/mobilnetfeatures/TensoresT/frame'+str(i+1)+'p.npy')
        #great_p.append(p)
        
        np.save('/home/oskkr/BIV2LAB/TensoresT/frame'+str(i+1)+'q',q)
        np.save('/home/oskkr/BIV2LAB/TensoresT/frame'+str(i+1)+'p',p)
        
    gc.collect()
    great_p=np.load('/home/oskkr/BIV2LAB/TensoresT/frame'+str(1)+'p.npy')
    d=great_p.shape[0]
    limy=great_p.shape[1]
    limx=great_p.shape[2]
    tr_list=cortar(trayectorias)
    nro=len(tr_list)  
    nro2=n_images*2
    Tensor=np.zeros((nro,n_images,d,d,3))
    tr_matrix=np.zeros((nro,nro2))
    x,y=tr_matrix.shape
    
    for i in range(0,x):
        k=2
        for j in range(int((tr_list[i][0]*2)),int(tr_list[i][1]*2)):
            tr_matrix[i][j]=tr_list[i][k]
            k=k+1
    #Obtengo mi matriz de trayectorias segun el caso
    for i in range(x):
      for j in range(0,y,2):
        if(tr_matrix[i][j]==0):pass
        else:
          if(tr_matrix[i,j]-area<=0 or tr_matrix[i,j]+area>=(limy-1)):
            tr_matrix[i,:]=np.zeros((tr_matrix[i].shape))
            break
          elif(tr_matrix[i,j+1]-area<=0 or tr_matrix[i,j+1]+area>=(limx-1)):
            tr_matrix[i,:]=np.zeros((tr_matrix[i].shape))
            break

    list_cov=[]
    for i in range(0,nro2,2):
        T_area=[]
        temp=int(i/2)+1
        for j in range(nro):
            if(tr_matrix[j,i]==0.):pass
            else:
                temp_area=[]
                nros=[]
                nros.append(j)
                nros.append(temp)
                y1=int(round(tr_matrix[j][i])-area)
                y2=int(round(tr_matrix[j][i])+area)
                x1=int(round(tr_matrix[j][i+1])-area)
                x2=int(round(tr_matrix[j][i+1])+area)
                temp_area.append(y1)
                temp_area.append(x1)
                temp_area.append(y2)
                temp_area.append(x2)

                T_area.append((temp_area,nros))
        great_p=np.load('/home/oskkr/BIV2LAB/TensoresT/frame'+str(temp)+'p.npy')
        great_q=np.load('/home/oskkr/BIV2LAB/TensoresT/frame'+str(temp)+'q.npy')    
        if(len(T_area)==0):pass
        else:
            Tensor=Tensor_covarianza(np.r_[great_q],np.r_[great_p],T_area,Tensor)
      
    #np.save('/content/drive/My Drive/BIVL2AB/Analisis de movimiento y vision por computador/Project/mobilnetfeatures/TensoresT2/SuperTensor',Tensor) 
    Covarianzas=covarianza(Tensor,area)

    return Covarianzas

## Parkinson

In [ ]:
import gc
from tqdm import tqdm
list_elements1,list_elements2=dirr_carps('/home/oskkr/BIV2LAB/Datasets/Parkinson/Trajectories/Ojos-PK-TrF/')
area=40
Tensor='/home/oskkr/BIV2LAB/TensoresT/'
path='/home/oskkr/BIV2LAB/FramesTemp/'

In [ ]:
for i in tqdm(range(10,13)):#len(list_elements1))):
    trayectorias=tr_text(list_elements1[i])
    path2=list_elements2[i][:-6]
    #print(path)
    #Eliminando frames de la carpeta 
    folder = path 

    for the_file in os.listdir(folder): 
        file_path = os.path.join(folder, the_file) 
        try: 
            if os.path.isfile(file_path): 
                os.unlink(file_path) 
         #elif os.path.isdir(file_path): shutil.rmtree(file_path) 
        except Exception as e: 
            print(e)
    #Eliminando Tensores anteriores        
    folder=Tensor

    for the_file in os.listdir(folder): 
      file_path = os.path.join(folder, the_file) 
      try: 
          if os.path.isfile(file_path): 
              os.unlink(file_path) 
        #elif os.path.isdir(file_path): shutil.rmtree(file_path) 
      except Exception as e: 
          print(e)

    #print('flag')
    
    #Obteniendo frames        
    print(list_elements2[i][:-6])
    vidcap = cv2.VideoCapture('/home/oskkr/BIV2LAB/Datasets/Parkinson/PK-ojos/'+list_elements2[i][:-6]+'.avi')
    success,image = vidcap.read()
    count = 1
    success = True
    while success:
        cv2.imwrite("/home/oskkr/BIV2LAB/FramesTemp/frame%d.jpg" % count, image)     # save frame as JPEG file      
        success,image = vidcap.read()
        #print('Read a new frame: ', success)
        count += 1
        
    
    gc.collect()
    #Extrayendo covarianzas    
    covs=video_covs(path,trayectorias,area,path2)
    #np.save('/content/drive/My Drive/BIVL2AB/Analisis de movimiento y vision por computador/Project/mobilnetfeatures/covarianzas/supertest',covs)
    np.save('/content/drive/My Drive/BIVL2AB/Analisis de movimiento y vision por computador/Project/parkinsonfeatures/TrajectoryCov/'+str(list_elements2[i][:-4]),covs)
    gc.collect()